<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/MidiTimingTools_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIDI Timing Tools

# Setup env


In [0]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 15
-rw------- 1 root root 10781 May 13 18:11 data_prep.py
drwx------ 2 root root  4096 May 10 13:31 __pycache__


## Auto reload module

I'm now using a module edited and saved to google drive which is automatically pushed to the cloud and made available to the colab env. Changes need time to propagate, and imports don't 'reimport' to load changes, so trying the following...

Note the code below is not very reliable, it seems to work occasionally, after some time, but haven't been able to work out the pattern to it. 

If in a hurry, brute force loading of changes by restarting the runtime.

In [0]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

In [0]:
# imports
import pandas as pd
import math

# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep



In [7]:
# testing auto reload of modules 
data_prep.test_function_call('bling')

test function called worked! :)  bling


In [0]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 200
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


## Test creating object from custom module

In [0]:
gmt = data_prep.GrooveMidiTools

In [18]:
file_name2 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid'
midi_file2 = data_prep.MIDI_File_Wrapper(file_name2, gmt.mappings)

FILE: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid
    tracks: [<midi track 'MIDI' 639 messages>]
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=588235 time=0>
    good instruments: 6, {36.0: 'Bass Drum 1', 38.0: 'Acoustic Snare', 42.0: 'Closed Hi Hat', 43.0: 'High Floor Tom', 46.0: 'Open Hi-Hat', 50.0: 'High Tom'}


... the above verifies I'm able to create custom objects from custom code, great!


In [19]:
file_name = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/1_funk-groove1_138_beat_4-4.mid'
midi_file = data_prep.MIDI_File_Wrapper(file_name, gmt.mappings)

FILE: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer5/eval_session/1_funk-groove1_138_beat_4-4.mid
    tracks: [<midi track 'Base Midi' 1037 messages>]
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    good instruments: 4, {36.0: 'Bass Drum 1', 38.0: 'Acoustic Snare', 42.0: 'Closed Hi Hat', 51.0: 'Ride Cymbal 1'}


# Create MidiTimingTools class

Been putting this together in a mix of actual code and pseudo code and jotting it down in Notepad++, pasting those notes here for reference.


SETUP PARAMS

```
- file header ticks per beat
- file meta microseconds per beat
- file header time sig. numerator
- file header time sig. denominator 
- last 'note-on' time in cumulative ticks
```

FUNCTIONS

```
- ticks_per_beat = <ticks per quarter beat> * ( 4/ < time sig denominator > )
- ticks_per_bar = < ticks_per_beat() >  * < time sig numerator >
- ticks_per_8 =  < file ticks per beat > / 2
- ticks_per_16 =  < file ticks per beat > / 4
- bars_in_file = math.ceil(last_note_on / ticks_per_bar()) # total bars to render, round up for whole bars
- ticks_in_file = bars_in_file() * ticks_per_bar() # total ticks to render (file_range)
- beats_in_file = bars_in_file() * < time sig numerator >
- bin_size = int(ticks_per_16()) # bucket size for quantizing


- get_bins(bin_size(), file_range=ticks_in_file()):   # my_bins
	return range(0 - (int(bin_size/ 2)), file_range + bin_size, bin_size)  

- get_beat_points(bin_size(),  file_range=ticks_in_file()):
	return range(0, file_range + bin_size, bin_size) 

# NOTE - don't think I actually need this at all
- calculated_bins(<cumulative_ticks_series>):
	return pd.cut(<cumulative_ticks_series>, bins=get_bins(), right=False)

# takes series with cumulative ticks since start of file for the 
# MIDI note_on events, and returns a series stating the centre
# of the beat for each given MIDI event
- assigned_beat_location(<cumulative_ticks_series>):
	return pd.cut(<cumulative_ticks_series> , bins=get_bins(), right=False, labels=get_beat_points())


- get_offsets(<cumulative_ticks_series>):
	
	my_beats = assigned_beat_location(<cumulative_ticks_series>)
	
	tmp_dict = dict(enumerate(my_beats.cat.categories))
	beat_centers = my_beats.cat.codes.map(tmp_dict)

	offsets = <cumulative_ticks_series> - beat_centers
	
	return offsets

```












In [0]:
import math

In [0]:


class MidiTimingTools:


  # all parameters required
  def __init__(label, file_ticks_per_beat, us_per_beat, time_sig_numerator, time_sig_denominator, last_note_on):
    self.label = label                                # pretty label, for handy reference
    self.file_ticks_per_beat = file_ticks_per_beat    # from MIDI file header
    self.time_sig_numerator = time_sig_numerator      #   "
    self.time_sig_denominator = time_sig_denominator  #   "
    self.us_per_beat = us_per_beat                    # from MIDI file meta message
    self.last_note_on = last_note_on                  # from MIDI file data


  # ts = time signature
  def ts_ticks_per_beat():
    return self.file_ticks_per_beat * ( 4/ self.time_sig_denominator )

  def ticks_per_bar():
    return ts_ticks_per_beat() * self.time_sig_numerator

  def ticks_per_8():
    return self.file_ticks_per_beat/ 2
    
  def ticks_per_16():
    return self.file_ticks_per_beat / 4

  # calculates total bars, round up for whole bars
  def bars_in_file():
    return math.ceil(self.last_note_on / ticks_per_bar()) 

  def ticks_in_file():
    return bars_in_file() * ticks_per_bar() # total ticks to render (file_range)

  def beats_in_file():
    return bars_in_file() * self.time_sig_numerator

  # bucket size for quantizing, set to 1/16 notes
  def bin_size():
    return int(ticks_per_16()) 

  def get_bins():   # my_bins
    my_bin_size = bin_size()
    file_range = ticks_in_file()
    return range(0 - (int(my_bin_size/ 2)), file_range + my_bin_size, my_bin_size)

  def get_beat_points():
    my_bin_size = bin_size()
    file_range = ticks_in_file()
    return range(0, file_range + bin_size, bin_size) 

  # NOTE - don't think I actually need this at all
  def calculated_bins(cumulative_ticks_series):
	  return pd.cut(cumulative_ticks_series, bins=get_bins(), right=False)

  # takes series with cumulative ticks since start of file for the 
  # MIDI note_on events, and returns a series stating the centre
  # of the beat for each given MIDI event
  def assigned_beat_location(cumulative_ticks_series):
	  return pd.cut(cumulative_ticks_series , bins=get_bins(), right=False, labels=get_beat_points())


  def get_offsets(cumulative_ticks_series):
    my_beats = assigned_beat_location(cumulative_ticks_series)
    tmp_dict = dict(enumerate(my_beats.cat.categories))
    beat_centers = my_beats.cat.codes.map(tmp_dict)
    offsets = cumulative_ticks_series - beat_centers
	
    return offsets

